In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import missingno as ms
import sklearn

In [2]:
data = pd.read_excel('Data_analysys_CR_discussed_1.xlsx')
data.head()

,type,Shap,"P1A, mol","P1ET, mol",CaC,Na,CaO,H2O,Reac,TEMP,RPM,Re_tyP1E,MF,P1ET mol,P1ET Conv %,P1A mol,P1A Conv %
0,LLD,P1El,0.0,0.0,0,0.0,0.0,0.0,14,300,50,Cust1,0.7,NaN,NaN,NaN,NaN
1,LLD,P1El,0.0,0.0,0,0.0,0.0,0.0,25,300,20,Cust1,1.2,NaN,NaN,NaN,NaN
2,LLD,P1El,0.0,0.0,0,0.0,0.0,0.0,14,350,50,Cust1,1.4,NaN,NaN,NaN,NaN
3,LLD,P1El,0.0,0.0,0,0.0,0.0,0.0,25,350,20,Cust1,19.0,NaN,NaN,NaN,NaN
4,LLD,P1El,0.0,0.0,0,0.0,0.0,0.0,25,370,60,Cust1,63.0,NaN,NaN,NaN,NaN


In [5]:
data.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   type         176 non-null    object 
 1   Shap         176 non-null    object 
 2   P1A, mol     176 non-null    float64
 3   P1ET, mol    176 non-null    float64
 4   CaC          176 non-null    int64  
 5   Na           175 non-null    float64
 6   CaO          172 non-null    float64
 7   H2O          176 non-null    float64
 8   Reac         176 non-null    int64  
 9   TEMP         176 non-null    int64  
 10  RPM          176 non-null    int64  
 11  Re_tyP1E     176 non-null    object 
 12  MF           141 non-null    float64
 13  P1ET mol     96 non-null     float64
 14  P1ET Conv %  81 non-null     float64
 15  P1A mol      11 non-null     float64
 16  P1A Conv %   11 non-null     float64
dtypes: float64(10), int64(4), object(3)
memory usage: 23.5+ KB


In [3]:
data['TEMP']= pd.to_numeric(data['TEMP'], errors='coerce')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   type         176 non-null    object 
 1   Shap         176 non-null    object 
 2   P1A, mol     176 non-null    float64
 3   P1ET, mol    176 non-null    float64
 4   CaC          176 non-null    int64  
 5   Na           175 non-null    float64
 6   CaO          172 non-null    float64
 7   H2O          176 non-null    float64
 8   Reac         176 non-null    int64  
 9   TEMP         176 non-null    int64  
 10  RPM          176 non-null    int64  
 11  Re_tyP1E     176 non-null    object 
 12  MF           141 non-null    float64
 13  P1ET mol     96 non-null     float64
 14  P1ET Conv %  81 non-null     float64
 15  P1A mol      11 non-null     float64
 16  P1A Conv %   11 non-null     float64
dtypes: float64(10), int64(4), object(3)
memory usage: 23.5+ KB


In [4]:
col1 = data.columns[:-4]
col1

Index(['type', 'Shap', 'P1A, mol', 'P1ET, mol', 'CaC', 'Na', 'CaO', 'H2O',
       'Reac', 'TEMP', 'RPM', 'Re_tyP1E', 'MF'],
      dtype='object')

In [5]:
data1=data[col1]

In [10]:
check_null=data1.isnull().sum()
check_null

type          0
Shap          0
P1A, mol      0
P1ET, mol     0
CaC           0
Na            1
CaO           4
H2O           0
Reac          0
TEMP          0
RPM           0
Re_tyP1E      0
MF           35
dtype: int64

In [6]:
data1=data1.dropna(axis=0,how="any")
data1.shape

(138, 13)

In [7]:
data1 = data1.drop(['CaC'],axis = 1)

In [8]:
check1_null=data1.isnull().sum()
check1_null

type         0
Shap         0
P1A, mol     0
P1ET, mol    0
Na           0
CaO          0
H2O          0
Reac         0
TEMP         0
RPM          0
Re_tyP1E     0
MF           0
dtype: int64

In [9]:
data1 = data1.drop(data1['Reac'][data1['Reac'] == 26].index,axis = 0)
data1 = data1[np.logical_and(data1['Re_tyP1E']!='Cust3' , data1['Re_tyP1E']!='Cust4')]
data1 = data1.drop(data1['H2O'][data1['H2O'] == 24].index,axis = 0)


In [13]:
[[j,data1[j].unique(),data1[j].value_counts() ]for j in data1.columns if data1[j].nunique()<20]

[['type',
  array(['LLD', 'P1P', 'Mix', 'LDP1E', 'P1E'], dtype=object),
  LLD      59
  P1E      32
  P1P      15
  LDP1E    12
  Mix       3
  Name: type, dtype: int64],
 ['Shap',
  array(['P1El', 'Fla', 'Mult. Fla'], dtype=object),
  P1El         87
  Mult. Fla    24
  Fla          10
  Name: Shap, dtype: int64],
 ['P1A, mol',
  array([0.        , 0.04418522, 0.08837045, 0.13255567]),
  0.000000    112
  0.044185      7
  0.088370      1
  0.132556      1
  Name: P1A, mol, dtype: int64],
 ['P1ET, mol',
  array([0.        , 0.02604167, 0.078125  , 0.01496347, 0.14963472,
         0.04401021, 0.04418522, 0.05208333]),
  0.000000    45
  0.026042    45
  0.044185    23
  0.078125     3
  0.044010     2
  0.014963     1
  0.149635     1
  0.052083     1
  Name: P1ET, mol, dtype: int64],
 ['H2O',
  array([ 0.,  8.,  5., 12.]),
  0.0     49
  12.0    33
  8.0     25
  5.0     14
  Name: H2O, dtype: int64],
 ['Reac',
  array([14, 25], dtype=int64),
  25    99
  14    22
  Name: Reac, dtype:

In [16]:
# data1['impurity'] = data1['P1A, mol']+data1['P1ET, mol']
# data1 = data1.drop(['P1A, mol','P1ET, mol'],axis = 1)
# data1

In [11]:
data1 = data1[np.logical_and(data1['Shap'] !='Fla, P1Ellets',data1['Shap'] != 'Pow')]

In [12]:
data1['Chem'] = data1['CaO']+data1['Na']
data1 = data1.drop(['CaO','Na'],axis = 1)
data1

,type,Shap,"P1A, mol","P1ET, mol",H2O,Reac,TEMP,RPM,Re_tyP1E,MF,Chem
0,LLD,P1El,0.000000,0.000000,0.0,14,300,50,Cust1,0.7,0.0
1,LLD,P1El,0.000000,0.000000,0.0,25,300,20,Cust1,1.2,0.0
2,LLD,P1El,0.000000,0.000000,0.0,14,350,50,Cust1,1.4,0.0
3,LLD,P1El,0.000000,0.000000,0.0,25,350,20,Cust1,19.0,0.0
4,LLD,P1El,0.000000,0.000000,0.0,25,370,60,Cust1,63.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
170,P1E,P1El,0.044185,0.026042,12.0,25,380,20,Cust1,32.0,2.5
171,P1E,P1El,0.088370,0.052083,12.0,25,370,20,Cust1,40.0,5.0
172,P1E,P1El,0.132556,0.078125,12.0,25,370,20,Cust1,100.0,7.5
173,Mix,P1El,0.000000,0.000000,0.0,14,300,50,Cust1,1.3,0.0


In [19]:
# data1 = data1.drop(['Chem'],axis = 1)

In [ ]:
data1['type']

In [25]:
data_p = data1

In [26]:
data_p = data_p[data_p['type'] == 'LLD']
data_p.shape

(59, 11)

In [27]:
data_p = data_p[data_p['Shap'] == 'P1El']
data_p.shape

(48, 11)

In [28]:
data_p = data_p[data_p['P1A, mol'] == 0]
data_p.shape

(48, 11)

In [29]:
data_p = data_p[data_p['Reac'] == 25]
data_p.shape

(34, 11)

In [30]:
data_p = data_p[data_p['Re_tyP1E'] == 'Cust1']
data_p.shape

(27, 11)

In [32]:
data_p = data_p[data_p['TEMP'] >= 350]
data_p.shape

(26, 11)

In [33]:
data_p = data_p[data_p['RPM'] >= 20]
data_p.shape

(26, 11)

In [34]:
data_p = data_p[np.logical_and(data_p['Chem'] >= 0,data_p['Chem']<= 2)]
data_p.shape

(26, 11)

In [35]:
data_p = data_p[np.logical_and(data_p['H2O'] >= 0,data_p['H2O']<= 12)]
data_p.shape

(26, 11)

In [36]:
data1.shape

(121, 11)